<a href="https://colab.research.google.com/github/alyssa-tsh/CS3244_ML_Project/blob/main/cs3244_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from functions_ver2 import data_pipeline as data_pipeline_v2, scaling_std
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
import numpy as np
from model_func import build_model, build_feature_selector
from visualize import plot_selected_features
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE, SelectFromModel
from xgboost import XGBClassifier

In [2]:
X_train_std, y_train, X_train_smote_std, y_train_smote, X_train_smotetomek_std, y_train_smotetomek, X_train_cc_std, y_train_cc, X_test_std, y_test = data_pipeline_v2('onehot')

Loading data
Splitting data
Total unique accounts: 45985. Starting to find cutoff point
Cutoff month where CDF reaches 80%: -10

=== Split based on CDF 80% cutoff ===
Cutoff month: -10 (10 months ago)
Old accounts (≤ month -10): 37,210 (80.9%)
New accounts (> month -10): 8,775 (19.1%)
Ratio (old/new): 4.2405
Splitting raw credit records
Cleaning old accounts credit records - [Length: 996586]


/workspaces/CS3244_ML_Project/functions_ver2.py:260: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({


Cleaning new accounts credit records - [Length: 51989]


/workspaces/CS3244_ML_Project/functions_ver2.py:260: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({
/workspaces/CS3244_ML_Project/functions_ver2.py:184: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dropped=df_sorted.groupby('id', group_keys=False).apply(keep_row)
/workspaces/CS3244_ML_Project/functions_ver2.py:184: FutureWarning: DataFr

Cleaning credit data completed
Splitting application dataset
Cleaning old accounts application records - [Length: (29264, 18)]
Cleaning new accounts appplication records, - [Length: (7193, 18)]
Encoding
Encoders: {'name_income_type': OneHotEncoder(handle_unknown='ignore', sparse_output=False), 'name_education_type': OneHotEncoder(handle_unknown='ignore', sparse_output=False), 'name_family_status': OneHotEncoder(handle_unknown='ignore', sparse_output=False), 'name_housing_type': OneHotEncoder(handle_unknown='ignore', sparse_output=False), 'occupation_type': OneHotEncoder(handle_unknown='ignore', sparse_output=False)}
Encoding type: onehot
Merging data
Engineering target variable to label data
Completed old accounts labelling
Completed new accounts labelling
Old accounts: (37210, 3)
New accounts: (8775, 3)
Old threshold: 0.20207707707707706
New threshold: 0.5786182336182336
Merging cleaned application and credit records
Train shape: (29264, 63)
Test shape: (7193, 63)
<class 'pandas.core.

In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def plot_selected_features(best_pipeline, X_columns, top_n=20):
    """
    Plots selected features and their importances for a given trained pipeline.
    """
    selector = best_pipeline.named_steps.get("feature_selector", None)
    model = best_pipeline.named_steps.get("classifier", None)

    plt.figure(figsize=(12, 5))
    
    # --- 1️⃣ Feature Selection Summary ---
    if selector and hasattr(selector, "get_support"):
        mask = selector.get_support()
        selected = np.array(X_columns)[mask]
        dropped = np.array(X_columns)[~mask]

        print(f"✅ Total features: {len(X_columns)} | Selected: {len(selected)} | Dropped: {len(dropped)}")
        print(f"🧩 Example selected features: {selected[:10]}")

        # Visualize counts
        plt.subplot(1, 2, 1)
        sns.barplot(
            x=["Selected", "Dropped"],
            y=[len(selected), len(dropped)],
            palette=["#00C49F", "#FF8042"]
        )
        plt.title("Feature Selection Summary")
        plt.ylabel("Number of Features")
    else:
        print("⚠️ No feature selector found; using all features.")
        selected = X_columns

    # --- 2️⃣ Feature Importances ---
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_
        feat_df = (
            pd.DataFrame({
                "feature": selected,
                "importance": importances
            })
            .sort_values("importance", ascending=False)
            .head(top_n)
        )

        plt.subplot(1, 2, 2)
        sns.barplot(
            data=feat_df,
            x="importance",
            y="feature",
            palette="viridis"
        )
        plt.title(f"Top {top_n} Important Features")
        plt.xlabel("Importance")
        plt.ylabel("")
    else:
        print("⚠️ Model does not expose `feature_importances_` attribute.")

    plt.tight_layout()
    plt.show()


In [62]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def build_feature_selector_xgb():
    """Feature selector using XGBoost importance."""
    return SelectFromModel(XGBClassifier(eval_metric="logloss", random_state=42), threshold='median')

def threshold_tuning(y_true, y_proba):
    """Find optimal threshold maximizing F1 score."""
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-6)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
    return best_threshold, f1_scores[best_idx]

def plot_feature_importance(model, feature_names, top_n=20):
    """Plot top N features selected by SelectFromModel."""
    selector = model.named_steps['feature_selector']
    clf = model.named_steps['classifier'].best_estimator_ 
    
    selected_mask = selector.get_support()
    selected_features = np.array(feature_names)[selected_mask]
    feature_importances = clf.feature_importances_[selected_mask]
    
    df = pd.DataFrame({
        'feature': selected_features,
        'importance': feature_importances
    }).sort_values('importance', ascending=False).head(top_n)
    
    plt.figure(figsize=(8,6))
    plt.barh(df['feature'][::-1], df['importance'][::-1], color='skyblue')
    plt.xlabel('Feature Importance')
    plt.title('Top Selected Features')
    plt.show()


def model_pipeline_xgb(X_train, y_train, X_test=None, y_test=None, n_splits=5, random_state=42):
    print("Starting XGBoost pipeline with hyperparameter tuning and feature selection...")
    
    # Compute imbalance weight
    pos_weight = sum(y_train == 0) / sum(y_train == 1)
    
    # Hyperparameter space for RandomizedSearch
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 0.1, 0.2, 0.5]
    }

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_models, acc_scores, f1_scores, roc_scores = [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Base classifier
        xgb = XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=random_state,
            scale_pos_weight=pos_weight
        )
        
        # Hyperparameter tuning
        rnd_search = RandomizedSearchCV(
            xgb, param_distributions=param_dist,
            n_iter=20, scoring='f1', cv=3, verbose=0, n_jobs=-1, random_state=random_state
        )
        
        pipeline = Pipeline([
            ('feature_selector', build_feature_selector_xgb()),
            ('classifier', rnd_search)
        ])
        
        pipeline.fit(X_tr, y_tr)
        
        y_proba = pipeline.predict_proba(X_val)[:,1]
        y_pred = pipeline.predict(X_val)
        
        acc_scores.append(accuracy_score(y_val, y_pred))
        f1_scores.append(f1_score(y_val, y_pred))
        roc_scores.append(roc_auc_score(y_val, y_proba))
        fold_models.append(pipeline)
        
        print(f"Fold {fold}: Accuracy={acc_scores[-1]:.3f}, F1={f1_scores[-1]:.3f}, ROC-AUC={roc_scores[-1]:.3f}")

    # Select best fold
    best_fold = int(np.argmax(roc_scores))
    best_model = fold_models[best_fold]
    print(f"\nBest model from Fold {best_fold+1} selected (ROC-AUC={roc_scores[best_fold]:.3f})")

    # Test evaluation
    results = {
        'cv_accuracy': np.mean(acc_scores),
        'cv_f1': np.mean(f1_scores),
        'cv_roc_auc': np.mean(roc_scores)
    }
    
    if X_test is not None and y_test is not None:
        y_proba_test = best_model.predict_proba(X_test)[:,1]
        best_threshold, best_f1_val = threshold_tuning(y_test, y_proba_test)
        y_pred_test = (y_proba_test > best_threshold).astype(int)
        
        results.update({
            'test_accuracy': accuracy_score(y_test, y_pred_test),
            'test_f1': f1_score(y_test, y_pred_test),
            'test_roc_auc': roc_auc_score(y_test, y_proba_test),
            'test_best_threshold': best_threshold,
            'test_f1_at_threshold': best_f1_val
        })
        
        print(f"\nTest Results → Accuracy={results['test_accuracy']:.3f}, "
              f"F1={results['test_f1']:.3f}, ROC-AUC={results['test_roc_auc']:.3f}, "
              f"Best Threshold={best_threshold:.2f}, F1@Threshold={best_f1_val:.3f}")
        
    return results, best_model


### Using SMOTE - has the most balanced performance - selected

In [63]:
best_model, metrics = model_pipeline_xgb(
    X_train_smote_std, y_train_smote, X_test_std, y_test, n_splits=5, random_state=42
)

Starting XGBoost pipeline with hyperparameter tuning and feature selection...
Fold 1: Accuracy=0.790, F1=0.762, ROC-AUC=0.859
Fold 2: Accuracy=0.805, F1=0.774, ROC-AUC=0.868
Fold 3: Accuracy=0.793, F1=0.762, ROC-AUC=0.864
Fold 4: Accuracy=0.793, F1=0.765, ROC-AUC=0.865
Fold 5: Accuracy=0.815, F1=0.784, ROC-AUC=0.873

Best model from Fold 5 selected (ROC-AUC=0.873)

Test Results → Accuracy=0.249, F1=0.228, ROC-AUC=0.541, Best Threshold=0.18, F1@Threshold=0.228


### Using SMOTE - Cluster Centeroids

In [15]:
results_df, X_train, y_train, X_test, y_test = model_pipeline(model_name="XGB", 
        X_train_full=X_train_cc_std, y_train_full=y_train_cc, X_test=X_test_std, y_test=y_test, target_col="label", random_state=42
    )


Training model: XGB Classifier using StratifiedKFold...


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:53:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 1: Accuracy=0.737, F1=0.760, ROC-AUC=0.825


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:53:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 2: Accuracy=0.760, F1=0.781, ROC-AUC=0.852


...

Fold 3: Accuracy=0.743, F1=0.760, ROC-AUC=0.847


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:53:34] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 4: Accuracy=0.729, F1=0.754, ROC-AUC=0.817


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:53:34] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 5: Accuracy=0.743, F1=0.767, ROC-AUC=0.842
Number of selected features: 28

Best model from Fold 2 selected (ROC-AUC=0.852)

Cross-Validation Summary → Accuracy: 0.742, F1: 0.764, ROC-AUC: 0.837

Evaluating best-performing model on unseen test set...
Test Results → Accuracy=0.426, F1=0.226, ROC-AUC=0.530


### Using SMOTE_TOMEK

In [6]:
results_df, X_train, y_train, X_test, y_test = model_pipeline(model_name="XGB", 
        X_train_full=X_train_smotetomek_std, y_train_full=y_train_smotetomek, X_test=X_test_std, y_test=y_test, target_col="label", random_state=42
    )


Training model: XGB Classifier using StratifiedKFold...


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:45:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 1: Accuracy=0.789, F1=0.759, ROC-AUC=0.857


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:45:05] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 2: Accuracy=0.815, F1=0.791, ROC-AUC=0.877


...

Fold 3: Accuracy=0.809, F1=0.781, ROC-AUC=0.877


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:45:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 4: Accuracy=0.786, F1=0.753, ROC-AUC=0.849


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [05:45:10] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 5: Accuracy=0.792, F1=0.762, ROC-AUC=0.862

Best model from Fold 3 selected (ROC-AUC=0.877)

Cross-Validation Summary → Accuracy: 0.798, F1: 0.769, ROC-AUC: 0.864

Evaluating best-performing model on unseen test set...
Test Results → Accuracy=0.688, F1=0.203, ROC-AUC=0.537


In [ ]:
best_model, metrics = model_pipeline_xgb(
        X_train_smotetomek_std,y_train_smotetomek, X_test_std, y_test, n_splits=5, random_state=42
    )

Starting XGBoost pipeline with hyperparameter tuning and feature selection...
Fold 1: Accuracy=0.791, F1=0.760, ROC-AUC=0.860
Fold 2: Accuracy=0.818, F1=0.793, ROC-AUC=0.881
Fold 3: Accuracy=0.811, F1=0.779, ROC-AUC=0.870
Fold 4: Accuracy=0.792, F1=0.763, ROC-AUC=0.857
Fold 5: Accuracy=0.792, F1=0.761, ROC-AUC=0.863

Best model from Fold 2 selected (ROC-AUC=0.881)

Test Results → Accuracy=0.275, F1=0.230, ROC-AUC=0.552, Best Threshold=0.17, F1@Threshold=0.231


ValueError: not enough values to unpack (expected 5, got 2)